# 符合Python风格的对象

In [ ]:
# 实现自定义类型的行为可以像内置类型那样自然

In [2]:
# __repr__ repr() 便于开发者理解的方式返回对象的字符串表示形式
# __str__ str() 便于用户理解的方式返回对象的字符串表示形式
# __bytes__ | bytes() 函数调用它获取对象的字节序列表示形式
# __format__ | format()

In [3]:
# !!! python 3 中，__repr__、__str__、和__format__方法都必须返回Unicode字符串（str 类型），只有__bytes__方法返回字节序列(bytes类型) !!!

In [15]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r},{!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                   bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({},{})'.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [4]:
# classmethod 与 staticmethod
# （看了很多，总结下来就是staticmethod就是普通函数，没啥luan用）
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [6]:
print (Demo.klassmeth())
print (Demo.klassmeth('spam'))
print (Demo.statmeth())
print (Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


In [7]:
# 格式化显示，内置format()和str.format()
brl = 1/2.43
print (format(brl, '0.4f'))
print ('1 BRL = {rate:0.2f} USD'.format(rate = brl))

0.4115
1 BRL = 0.41 USD


In [10]:
# b和x分别表示二进制和十六进制的int类型
# f表示小数形式的float类型，而%表示百分数形式
print (format(42, 'b'))
print (format(2/3, '0.1%'))

101010
66.7%


In [12]:
from datetime import datetime
now = datetime.now()
print (now)
print (format(now, '%H:%M:%S'))
print ("It's now {:%I:%M:%S}".format(now))

2019-11-19 14:11:09.997845
14:11:09
It's now 02:11:09


In [17]:
# 类没有定义 __format__方法，从object继承的方法会返回str(my_object)
v1 = Vector2d(3,4)
print (format(v1, '0.2f'))

(3.00,4.00)


In [ ]:
# 可散列的Vector2d,必须使用__hash__方法

In [24]:
class Vecotr2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.__x, self.__y))

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [26]:
v1 = Vecotr2d(3, 4)
v2 = Vecotr2d(4, 3)

In [27]:
print (hash(v1), hash(v2))

7 7


In [29]:
# Python的私有属性和“受保护的”属性
# __pname python会把属性名存入实例的__dict__属性中,而且会在前面加上一个下划线和类名
print (v1.__dict__)

{'_Vecotr2d__x': 3.0, '_Vecotr2d__y': 4.0}


In [31]:
# 可以通过这种方式故意修改属性
v1._Vecotr2d__x = 4

In [ ]:
# 使用__slots__类属性节省空间

class Vector2d:
    __slots__ = ('__x', '__y')
    typecode = 'd'

    # 下面是各个方法

# __slots__的问题
# 1 每个子类都要定义__slots__属性，因为解释器会忽略继承的__slots__属性
# 2 实例只能拥有__slots__中列出的属性，除非把'__dict__'加入__slots__中（这样做就失去了节省内存的功效）
# 3 如果不把'__weakref__'加入__slots__，实例就不能作为弱引用的目标